# Cracking an Old Cipher

Encrypt and decrypt something using the vigenere cipher and a key. Then, we try to crack the cypher and find the key.

Frequency table provided by [this study](https://link.springer.com/content/pdf/10.3758/BF03195586.pdf)

In [1]:
import math as mt
import numpy as np
import matplotlib
import ipywidgets as widg
matplotlib.rcParams['figure.figsize'] = [12, 8]
from matplotlib import pyplot as plt

# Helper functions
symbol_space = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789.,!?&()"\':;- \n'
to_ints = lambda text: np.array([ symbol_space.index(c) for c in text ], dtype=np.int8)
to_chrs = lambda ints: ''.join([ symbol_space[i] for i in ints ])

def extend_to_fit(key, text):
    """
    Extend key by repitition to
    match length of key to length
    of text
    
    :param key: key to extend
    :param text: text to fit to
    
    :return: extended key
    """
    keys = key * ( mt.ceil(len(plaintext)/len(key)) )
    return keys[:len(plaintext)]

Our plaintext and key

In [35]:
plaintext = """Did you ever hear the tragedy of Darth Plagueis The Wise? 
I thought not. It's not a story the Jedi would tell you. It's a Sith 
legend. Darth Plagueis was a Dark Lord of the Sith, so powerful and 
so wise he could use the Force to influence the midichlorians to create 
life. He had such a knowledge of the dark side that he could even keep 
the ones he cared about from dying. The dark side of the Force is a 
pathway to many abilities some consider to be unnatural. He became so 
powerful. the only thing he was afraid of was losing his power, which 
eventually, of course, he did. Unfortunately, he taught his apprentice 
everything he knew, then his apprentice killed him in his sleep. Ironic. 
He could save others from death, but not himself."""

key = 'itisnotpossibletolearnthisfromajedi'
print(len(key))

35


Encrypt it using vigenere cipher

In [3]:
# Convert plaintext and key to integers
plaintext_i = to_ints(plaintext)

# Extend key to fit plaintext
keys_i = to_ints(extend_to_fit(key, plaintext))

# Encrypt plaintext using key into cyphertext
ciphertext = to_chrs((plaintext_i + keys_i) % len(symbol_space))

# Print cyphertext
print('Ciphertext:')
print()
print(ciphertext)

Ciphertext:

LBlqLCNnsNwz
sitFjxhvlMyiyjuMkoocGizMpq2ztvIwAA
4lxm7msvcrgQqyyCGgqxbvwM'qVHnHmFGB
lcLHzvypGAlg1juwkwxyolgMmDymRDIeqQufwrojWiKurgtwlvBp.hHdzBAg7yozJsAKgxlwrojHaIxrSwJipCr Clhg0BBzamLDmHGEfCjNzjenulszwqBzGq qibkwNtvlILtmLzm
QsKqpctFlBunDzvBoehxkmgFqvvqAACJAioDcMCjgrvnMlgrqztq.hLhgptlqFIvwmsqsozAEsokepByfBzjprmrtcvqlxgLuoMnvwqkpFpwmpzeElDlmHdqHtehsqmArpwlqtGsvqiczyMmqvoDlwFqFldm5hncgizDgKvrxnCxqBipcYCCgepvLfiqeGoFhFeBgBHgEnBRnotAtjEmxGjwoDrrjwFxzrqrhxrgjxgMABtIIJst,jLxmmicrzxfAGdqDAwnviCtfgLusrDBDQgusmGujlepJtzgskIoudhsigEtAqyCLxByqpjDcICHirelPoqumpnqvnrwCiEtQamHumuGCsDigmsi uvw"g!swCDtDrdBmEGflvxnHsMoiEcAwDcaGCKlvLntsk
nzhzGMpAAurwsqCvfH(rHsinpuBzgsuGFqnCmfmgDqDyswnvAEgjycAwDcsCrxw'qNICzil&bhPxguBIEsmKsDfjsMvpvspsKvuqivoFh;ceCBrvGGmAxAKwtg-


Now let's try decrypting the text using our key. This is the reverse of the encryption process

In [4]:
# Convert ciphertext and key to integers
# Extend key to fit ciphertext
ciphertext_i = to_ints(ciphertext)

# Decrypt ciphertext using key into plaintext
dplaintext = to_chrs((ciphertext_i - keys_i) % len(symbol_space))

# Print plaintext
print(dplaintext)

Did you ever hear the tragedy of Darth Plagueis The Wise? 
I thought not. It's not a story the Jedi would tell you. It's a Sith 
legend. Darth Plagueis was a Dark Lord of the Sith, so powerful and 
so wise he could use the Force to influence the midichlorians to create 
life. He had such a knowledge of the dark side that he could even keep 
the ones he cared about from dying. The dark side of the Force is a 
pathway to many abilities some consider to be unnatural. He became so 
powerful. the only thing he was afraid of was losing his power, which 
eventually, of course, he did. Unfortunately, he taught his apprentice 
everything he knew, then his apprentice killed him in his sleep. Ironic. 
He could save others from death, but not himself.


If we don't have the key, we theoretically cannot decypher the cyphertext... or can we?

The only difficult part is to find the length of the cipher. We can find a set of possible lengths, and we'll have to brute force each one

In [5]:
# Convert cyphertext to integers
ciphertext_i = to_ints(ciphertext)

# Convert to list of n-graphs
n = 2
pad = np.zeros((n - len(ciphertext_i) % n,), dtype=np.int32)
pad_ciphertext_i = np.append(ciphertext_i, pad)
ngraphs_i = pad_ciphertext_i.reshape(-1, n)

# Gather unique distances between like ngraphs
num_ngraphs = ngraphs_i.shape[0]
distances = np.array([], dtype=np.int32)
for i in range(num_ngraphs):
    indices = np.argwhere((ngraphs_i == ngraphs_i[i,:]).all(axis=1)).reshape(-1)
    if len(indices) > 1:
        distances = np.concatenate((distances, np.diff(indices)))
distances = np.unique(distances)
distances

# List all prime numbers up to max distance
max_distance = distances.max()
primes = [2]
for i in range(2, max_distance):
    for prime in primes:
        if i % prime == 0:
            break
    else:
        primes.append(i)
        
# Generate list of factors for each distance
factors = []
for distance in distances:
    dfactors = set()
    for prime in primes:
        ffactor = prime
        while ffactor < distance and distance % ffactor == 0:
            dfactors.add(ffactor)
            dfactors.add(distance // ffactor)
            ffactor *= prime
    factors.extend(list(dfactors))

# List factors sorted by most occurances
ufactors, counts = np.unique(factors, return_counts=True)
likely_lengths = ufactors[np.flip(np.argsort(counts))]
print(likely_lengths)

[  2   3   7   4   5   9  35  19  18   8  11  70  38  29  21  37  58  14
  10   6  17  16  26  13  20  12  23  25 140  28  72 117 105 103  95  87
  78  76  74  67 118  59  56  48  42  41  40  36  31  30]


Frequency analysis